# Credit Card Consumption Prediction
## Business Problem
Understanding consumer credit card spending is crucial for banks to develop personalized marketing campaigns and manage credit risk. This project aims to predict customers' credit card consumption for the next three months based on past spending behavior and demographic information.

## Objectives
- Perform **Exploratory Data Analysis (EDA)** to identify trends and missing values.
- **Impute missing values** using appropriate statistical techniques.
- Build a **predictive model** to forecast credit card consumption.
- Evaluate model performance using **Root Mean Square Percentage Error (RMSPE)**.

In [1]:
## Importing neccessary library
import pandas as pd
import numpy as np
import scipy.stats as stats
import itertools

In [2]:
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [3]:
# Importing all 3 data sets
cust_con = pd.read_excel(r"C:\MACHINE LEARNING\PROJECT\11. Capstone Case Study - Predict Cred Card Consumption (1)\CreditConsumptionData.xlsx")
cust_behav = pd.read_excel(r"C:\MACHINE LEARNING\PROJECT\11. Capstone Case Study - Predict Cred Card Consumption (1)\CustomerBehaviorData.xlsx")
cust_demo = pd.read_excel(r"C:\MACHINE LEARNING\PROJECT\11. Capstone Case Study - Predict Cred Card Consumption (1)\CustomerDemographics.xlsx")

In [252]:
cust_demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            20000 non-null  int64  
 1   account_type                  19999 non-null  object 
 2   gender                        19999 non-null  object 
 3   age                           20000 non-null  int64  
 4   Income                        19999 non-null  object 
 5   Emp_Tenure_Years              20000 non-null  float64
 6   Tenure_with_Bank              20000 non-null  int64  
 7   region_code                   19999 non-null  float64
 8   NetBanking_Flag               20000 non-null  int64  
 9   Avg_days_between_transaction  19997 non-null  float64
dtypes: float64(3), int64(4), object(3)
memory usage: 1.5+ MB


#####  Data overview

In [254]:
cust_demo.head()

,ID,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,19427,current,M,63,MEDIUM,30.1,10,628.0,1,5.0
1,16150,current,M,36,MEDIUM,14.4,10,656.0,0,12.0
2,11749,current,F,28,MEDIUM,4.8,10,314.0,1,13.0
3,11635,current,M,32,MEDIUM,9.6,2,614.0,1,19.0
4,8908,current,M,32,HIGH,12.0,7,750.0,1,18.0


In [25]:
cust_behav.head()

,ID,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,17051,3412.60,2909.34,1077.36,3084.50,4615.97,6693.0,19.0,10,2,...,1,108,97916.0,45253.64,47635.00,2,18,19947.0,Y,3448.84
1,11491,18133.76,3877.00,6123.78,5597.43,64620.00,6117.0,4.0,60,1,...,1,4,25125.0,12912.33,18369.00,3,49,20585.0,Y,3812.69
2,7433,6363.65,735.49,8799.00,13768.00,38266.00,2638.0,2.0,83,47,...,70,11,38546.0,93567.51,26713.00,47,49,13950.0,Y,9432.90
3,14606,12765.66,4429.16,16745.86,6360.00,29063.97,3711.0,12.0,2,16,...,43,1,180315.0,176394.48,35062.74,3,37,37018.0,Y,144.61
4,8381,27819.70,1944.00,7006.50,2228.50,1096.25,1065.0,15.0,13,61,...,2,14,13480.0,3167.00,19056.00,2,15,24274.0,Y,1887.89


In [38]:
cust_con.head()

,ID,cc_cons
0,12554,20014.0
1,17645,10173.0
2,7604,16095.0
3,1590,7707.0
4,16556,96408.0


## Exploratory Data Analysis (EDA)
- Check for missing values and handle them appropriately.
- Identify outliers and distribution of key variables.
- Analyze spending patterns across demographics.

###### Merging Data

In [4]:
cust_bhave_demo =pd.merge(left=cust_demo, right=cust_behav, on=["ID","ID"], how="inner")

In [5]:
customer = pd.merge(left=cust_bhave_demo, right=cust_con, on=["ID","ID"], how="inner")

In [195]:
customer

,ID,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction,...,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons
0,19427,current,M,63,MEDIUM,30.1,10,628.0,1,5.0,...,14,20770.0,44884.90,369000.75,40,96,46088.0,Y,2646.72,NaN
1,16150,current,M,36,MEDIUM,14.4,10,656.0,0,12.0,...,63,78627.0,91073.84,243182.32,7,12,17953.0,Y,5469.79,NaN
2,11749,current,F,28,MEDIUM,4.8,10,314.0,1,13.0,...,82,1260.0,96552.00,35467.00,16,42,41121.0,Y,7207.85,NaN
3,11635,current,M,32,MEDIUM,9.6,2,614.0,1,19.0,...,78,23332.0,18250.00,87204.35,2,10,32003.0,Y,591.34,NaN
4,8908,current,M,32,HIGH,12.0,7,750.0,1,18.0,...,171,41672.0,83525.75,83992.00,21,12,9626.0,Y,2621.39,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1270,current,F,66,MEDIUM,32.2,6,354.0,1,7.0,...,8,24438.0,153772.21,31284.00,3,38,42872.0,Y,6334.20,NaN
19996,15992,current,M,53,MEDIUM,19.8,5,809.0,0,14.0,...,11,213000.0,18098.47,93108.00,43,11,36747.0,Y,102216.02,NaN
19997,7081,current,F,66,MEDIUM,18.4,1,466.0,0,10.0,...,171,34327.0,32198.63,46800.00,3,38,18810.0,Y,1930.19,NaN
19998,6821,current,M,32,LOW,6.0,8,619.0,0,17.0,...,82,51929.0,27334.14,20201.00,47,80,11443.0,Y,24499.91,NaN


##### Data Cleaning And Pre Modeling

In [45]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            20000 non-null  int64  
 1   account_type                  19999 non-null  object 
 2   gender                        19999 non-null  object 
 3   age                           20000 non-null  int64  
 4   Income                        19999 non-null  object 
 5   Emp_Tenure_Years              20000 non-null  float64
 6   Tenure_with_Bank              20000 non-null  int64  
 7   region_code                   19999 non-null  float64
 8   NetBanking_Flag               20000 non-null  int64  
 9   Avg_days_between_transaction  19997 non-null  float64
 10  cc_cons_apr                   20000 non-null  float64
 11  dc_cons_apr                   20000 non-null  float64
 12  cc_cons_may                   19999 non-null  float64
 13  d

In [46]:
customer.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,20000.0,9999.500000,5.773647e+03,0.00,4999.750,9999.5000,14999.2500,19999.00
age,20000.0,47.672300,3.035977e+01,25.00,33.000,40.0000,53.0000,223.00
Emp_Tenure_Years,20000.0,15.217250,2.048145e+01,0.50,5.100,9.9000,18.0000,203.00
Tenure_with_Bank,20000.0,5.496050,2.870452e+00,1.00,3.000,6.0000,8.0000,10.00
region_code,19999.0,598.959348,2.201052e+02,123.00,424.000,623.0000,799.0000,974.00
NetBanking_Flag,20000.0,0.718000,4.499846e-01,0.00,0.000,1.0000,1.0000,1.00
Avg_days_between_transaction,19997.0,11.044757,5.480552e+00,2.00,6.000,11.0000,16.0000,20.00
cc_cons_apr,20000.0,17856.990922,2.515888e+04,253.00,4377.000,10214.6425,21189.0000,441031.33
dc_cons_apr,20000.0,8130.453414,1.457219e+04,19.35,1624.650,3964.7500,9339.0200,229255.00
cc_cons_may,19999.0,16188.364717,2.651068e+04,157.00,3635.910,8635.2300,18655.3050,1117246.31


In [ ]:
# cc_cons_x and cc_cons_y have outliers

In [47]:
customer.isnull().sum()

ID                                 0
account_type                       1
gender                             1
age                                0
Income                             1
Emp_Tenure_Years                   0
Tenure_with_Bank                   0
region_code                        1
NetBanking_Flag                    0
Avg_days_between_transaction       3
cc_cons_apr                        0
dc_cons_apr                        0
cc_cons_may                        1
dc_cons_may                        0
cc_cons_jun                        0
dc_cons_jun                        1
cc_count_apr                       1
cc_count_may                       0
cc_count_jun                       0
dc_count_apr                       0
dc_count_may                       0
dc_count_jun                       0
card_lim                           0
personal_loan_active               0
vehicle_loan_active                0
personal_loan_closed               1
vehicle_loan_closed                0
i

In [6]:
## dropping loan_enq , since it is a categorical variable and all values are same "Y"
customer.drop(columns='loan_enq', inplace=True)

In [148]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 48 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            20000 non-null  int64  
 1   account_type                  19999 non-null  object 
 2   gender                        19999 non-null  object 
 3   age                           20000 non-null  int64  
 4   Income                        19999 non-null  object 
 5   Emp_Tenure_Years              20000 non-null  float64
 6   Tenure_with_Bank              20000 non-null  int64  
 7   region_code                   19999 non-null  float64
 8   NetBanking_Flag               20000 non-null  int64  
 9   Avg_days_between_transaction  19997 non-null  float64
 10  cc_cons_apr                   20000 non-null  float64
 11  dc_cons_apr                   20000 non-null  float64
 12  cc_cons_may                   19999 non-null  float64
 13  d

In [7]:
# Splitting Continuous Variables and Categorical Variables
customer.select_dtypes(include=['int64','float64']).columns

Index(['ID', 'age', 'Emp_Tenure_Years', 'Tenure_with_Bank', 'region_code',
       'NetBanking_Flag', 'Avg_days_between_transaction', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun', 'card_lim',
       'personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed',
       'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'emi_active', 'cc_cons'],
      dtype='object')

In [8]:
continuous = ['age', 'Emp_Tenure_Years', 'Tenure_with_Bank','Avg_days_between_transaction', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun','investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun','emi_active', 'cc_cons']
cat_col = ['account_type', 'gender','Income','region_code','NetBanking_Flag']

In [9]:
cont_customer = customer.loc[:,continuous]

In [10]:
cat_customer = customer.loc[:,cat_col]

In [ ]:
# Creating function for summary and ouliers

In [13]:
def cont_var_summary( x ):
    # frequancy and missings
    x_total = x.shape[0]
    x_miss = x.isna().sum()
    perc_miss = x_miss * 100 / x_total
    
    # outliers 
    q1 = x.quantile(0.25)
    q3 = x.quantile(0.75)
    iqr = q3 - q1
    lc_iqr = q1 - 1.5 * iqr
    uc_iqr = q3 + 1.5 * iqr
    
    return pd.Series( [ x.dtype, x.nunique(), x_total, x.count(), x_miss, perc_miss,
                       x.sum(), x.mean(), x.std(), x.var(), 
                       lc_iqr, uc_iqr, 
                       x.min(), x.quantile(0.01), x.quantile(0.05), x.quantile(0.10), 
                       x.quantile(0.25), x.quantile(0.5), x.quantile(0.75), 
                       x.quantile(0.90), x.quantile(0.95), x.quantile(0.99), x.max() ], 
                     
                    index = ['dtype', 'cardinality', 'x_tot', 'x', 'nmiss', 'perc_miss',
                             'sum', 'mean', 'std', 'var',
                        'lc_iqr', 'uc_iqr',
                        'min', 'p1', 'p5', 'p10', 'p25', 'p50', 'p75', 'p90', 'p95', 'p99', 'max']) 

In [14]:
cont_customer.apply(cont_var_summary)

,age,Emp_Tenure_Years,Tenure_with_Bank,Avg_days_between_transaction,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,...,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,cc_cons
dtype,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,...,int64,int64,float64,float64,float64,int64,int64,float64,float64,float64
cardinality,43,270,10,19,8459,3861,8504,4053,7347,3019,...,31,57,5299,8632,8311,39,66,5833,5940,12260
x_tot,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,...,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000
x,20000,20000,20000,19997,20000,20000,19999,20000,20000,19999,...,20000,20000,20000,20000,20000,20000,20000,20000,19999,15000
nmiss,0,0,0,3,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,5000
perc_miss,0.0,0.0,0.0,0.015,0.0,0.0,0.005,0.0,0.0,0.005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,25.0
sum,953446,304345.0,109921,220862.0,357139818.44,162609068.27,323751105.97,154048543.82,242716120.41,142187057.12,...,238380,819497,1164159702.76,1562455943.19,1576338552.15,398109,890582,1056996951.42,307961976.6,827365948.0
mean,47.6723,15.21725,5.49605,11.044757,17856.990922,8130.453414,16188.364717,7702.427191,12135.80602,7109.708341,...,11.919,40.97485,58207.985138,78122.797159,78816.927608,19.90545,44.5291,52849.847571,15398.868773,55157.729867
std,30.359771,20.481455,2.870452,5.480552,25158.878234,14572.188829,26510.681596,17892.7318,18790.984789,13313.935609,...,16.85576,40.561202,208591.362703,244898.308732,156987.241764,27.055323,35.080453,134317.525107,120254.075769,97670.972252
var,921.715698,419.489979,8.239496,30.036454,632969153.988079,212348687.260089,702816238.690255,320149851.281652,353101109.329598,177260881.388224,...,284.116645,1645.211078,43510356594.345634,59975181619.710663,24644994076.810459,731.99051,1230.638185,18041197550.945358,14461042739.004244,9539618820.721489


In [15]:
customer[['account_type', 'gender','Income']].describe()

,account_type,gender,Income
count,19999,19999,19999
unique,2,2,3
top,current,M,MEDIUM
freq,16925,17287,12774


##### Outlier Treatment

In [16]:
cont_customer.apply( lambda x: x.clip(lower = x.quantile(0.01),upper = x.quantile(0.99),inplace=True))

age                             None
Emp_Tenure_Years                None
Tenure_with_Bank                None
Avg_days_between_transaction    None
cc_cons_apr                     None
dc_cons_apr                     None
cc_cons_may                     None
dc_cons_may                     None
cc_cons_jun                     None
dc_cons_jun                     None
cc_count_apr                    None
cc_count_may                    None
cc_count_jun                    None
dc_count_apr                    None
dc_count_may                    None
dc_count_jun                    None
investment_1                    None
investment_2                    None
investment_3                    None
investment_4                    None
debit_amount_apr                None
credit_amount_apr               None
debit_count_apr                 None
credit_count_apr                None
max_credit_amount_apr           None
debit_amount_may                None
credit_amount_may               None
c

In [266]:
cont_customer.apply(cont_var_summary)

,age,Emp_Tenure_Years,Tenure_with_Bank,Avg_days_between_transaction,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,...,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,cc_cons
dtype,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,...,int64,int64,float64,float64,float64,int64,int64,float64,float64,float64
cardinality,41,246,10,19,8302,3789,8345,3972,7205,2951,...,30,56,5196,8476,8150,39,65,5718,5832,11990
x_tot,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000,...,20000,20000,20000,20000,20000,20000,20000,20000,20000,20000
x,20000,20000,20000,19997,20000,20000,19999,20000,20000,19999,...,20000,20000,20000,20000,20000,20000,20000,20000,19999,15000
nmiss,0,0,0,3,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,5000
perc_miss,0.0,0.0,0.0,0.015,0.0,0.0,0.005,0.0,0.0,0.005,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,25.0
sum,953579,294931.2,109921,220862.0,346146525.37,152855585.51,311485406.43,143831037.93,232619464.114999,135690695.56,...,238356,819575,1012022855.42,1425114478.53,1456158777.81,398109,889961,956316532.8,201720730.94,826365410.0
mean,47.67895,14.74656,5.49605,11.044757,17307.326269,7642.779275,15575.049074,7191.551896,11630.973206,6784.874022,...,11.9178,40.97875,50601.142771,71255.723927,72807.938891,19.90545,44.49805,47815.82664,10086.540874,55091.027333
std,30.311294,17.040411,2.870452,5.480552,21180.687109,10456.109254,20400.815735,10328.490113,14689.082389,10609.560566,...,16.851589,40.557309,59226.918134,81596.204599,85133.417139,27.055323,35.006033,57339.279089,21303.303095,97429.489575
var,918.774516,290.375592,8.239496,30.036454,448621506.394323,109330220.730953,416193282.644561,106677708.019133,215769141.419008,112562775.406872,...,283.976042,1644.895343,3507827831.614031,6657940604.970504,7247698713.828243,731.99051,1225.422367,3287792926.408136,453830722.744924,9492505438.819309


In [159]:
# Seperating cc-cons from data, to carry out missing value imputation

In [17]:
cont_cust1 = cont_customer[['age', 'Emp_Tenure_Years', 'Tenure_with_Bank','Avg_days_between_transaction', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun','investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun','emi_active']]

In [18]:
cont_cust2 = cont_customer['cc_cons']

##### Mising value Imputation

In [19]:
cont_col1 = ['age', 'Emp_Tenure_Years', 'Tenure_with_Bank','Avg_days_between_transaction', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun','investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun','emi_active']

In [20]:
for i in cont_col1:
    cont_cust1[i] = cont_cust1[i].fillna(cont_cust1[i].mean())

C:\Users\Tarun\AppData\Local\Temp\ipykernel_10116\78284495.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cont_cust1[i] = cont_cust1[i].fillna(cont_cust1[i].mean())


In [21]:
cont_cust1.isnull().sum()

age                             0
Emp_Tenure_Years                0
Tenure_with_Bank                0
Avg_days_between_transaction    0
cc_cons_apr                     0
dc_cons_apr                     0
cc_cons_may                     0
dc_cons_may                     0
cc_cons_jun                     0
dc_cons_jun                     0
cc_count_apr                    0
cc_count_may                    0
cc_count_jun                    0
dc_count_apr                    0
dc_count_may                    0
dc_count_jun                    0
investment_1                    0
investment_2                    0
investment_3                    0
investment_4                    0
debit_amount_apr                0
credit_amount_apr               0
debit_count_apr                 0
credit_count_apr                0
max_credit_amount_apr           0
debit_amount_may                0
credit_amount_may               0
credit_count_may                0
debit_count_may                 0
max_credit_amo

In [22]:
cat_customer.isnull().sum()

account_type       1
gender             1
Income             1
region_code        1
NetBanking_Flag    0
dtype: int64

In [23]:
for i in cat_col:
    cat_customer[i] = cat_customer[i].fillna(cat_customer[i].mode().iloc[0])

In [24]:
cat_customer.isnull().sum()

account_type       0
gender             0
Income             0
region_code        0
NetBanking_Flag    0
dtype: int64

##### Dummy Variables

In [275]:
cat_customer.Income.value_counts()

Income
MEDIUM    12775
LOW        5612
HIGH       1613
Name: count, dtype: int64

In [25]:
cat_customer['Income'] = pd.Series(np.where(cat_customer.Income=="LOW",1,np.where(cat_customer.Income=="Medium",2,3)))

In [26]:
cat_customer

,account_type,gender,Income,region_code,NetBanking_Flag
0,current,M,3,628.0,1
1,current,M,3,656.0,0
2,current,F,3,314.0,1
3,current,M,3,614.0,1
4,current,M,3,750.0,1
...,...,...,...,...,...
19995,current,F,3,354.0,1
19996,current,M,3,809.0,0
19997,current,F,3,466.0,0
19998,current,M,1,619.0,0


In [27]:
cat_customer.account_type.value_counts()

account_type
current    16926
saving      3074
Name: count, dtype: int64

In [28]:
cat_customer.gender.value_counts()

gender
M    17288
F     2712
Name: count, dtype: int64

In [29]:
for i in ['account_type', 'gender']:
    cols = pd.get_dummies(cat_customer[i], prefix=i, drop_first=True).astype(int)  # Convert to 0/1
    cat_customer = pd.concat([cat_customer, cols], axis=1)
    cat_customer.drop(i, axis=1, inplace=True)

In [30]:
cat_customer

,Income,region_code,NetBanking_Flag,account_type_saving,gender_M
0,3,628.0,1,0,1
1,3,656.0,0,0,1
2,3,314.0,1,0,0
3,3,614.0,1,0,1
4,3,750.0,1,0,1
...,...,...,...,...,...
19995,3,354.0,1,0,0
19996,3,809.0,0,0,1
19997,3,466.0,0,0,0
19998,1,619.0,0,0,1


In [31]:
cat_customer.region_code.value_counts()

region_code
249.0    509
683.0    380
867.0    369
748.0    356
355.0    354
        ... 
608.0      1
436.0      1
484.0      1
398.0      1
857.0      1
Name: count, Length: 210, dtype: int64

In [32]:
cat_customer.NetBanking_Flag.value_counts()

NetBanking_Flag
1    14360
0     5640
Name: count, dtype: int64

In [33]:
for i in ['region_code', 'NetBanking_Flag']:
    cat_customer[i] = cat_customer[i].astype('category')  # Convert to category
    cols = pd.get_dummies(cat_customer[i], prefix=i, drop_first=True).astype(int)  # Convert to 0/1
    cat_customer = pd.concat([cat_customer, cols], axis=1)
    cat_customer.drop(i, axis=1, inplace=True)

In [34]:
cat_customer

,Income,account_type_saving,gender_M,region_code_233.0,region_code_243.0,region_code_244.0,region_code_245.0,region_code_247.0,region_code_248.0,region_code_249.0,...,region_code_958.0,region_code_963.0,region_code_964.0,region_code_965.0,region_code_966.0,region_code_968.0,region_code_969.0,region_code_970.0,region_code_974.0,NetBanking_Flag_1
0,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19996,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
cat_customer.columns = cat_customer.columns.str.replace('.','_')

In [287]:
cat_customer

,Income,account_type_saving,gender_M,region_code_233_0,region_code_243_0,region_code_244_0,region_code_245_0,region_code_247_0,region_code_248_0,region_code_249_0,...,region_code_958_0,region_code_963_0,region_code_964_0,region_code_965_0,region_code_966_0,region_code_968_0,region_code_969_0,region_code_970_0,region_code_974_0,NetBanking_Flag_1
0,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19996,3,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Combining The data

In [36]:
clean_customer = pd.concat([cat_customer,cont_cust1],axis=1)

In [37]:
clean_customer = pd.concat([clean_customer, cont_cust2],axis=1)

In [ ]:
# Seperating the data which is available and which we have to predict

In [38]:
clean_customer_avail = clean_customer[~clean_customer.cc_cons.isna()]

In [39]:
clean_customer_topred = clean_customer[clean_customer.cc_cons.isna()]

In [40]:
feature_columns = clean_customer_avail.columns.difference(['cc_cons'])

In [41]:
train_X, test_X, train_Y, test_Y = train_test_split(clean_customer_avail[feature_columns],clean_customer_avail['cc_cons'],test_size=0.3, random_state = 2)

In [ ]:
# importing 

In [42]:
from sklearn.tree import DecisionTreeRegressor

In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [44]:
params = {'criterion':["squared_error","friedman_mse","absolute_error"],
    'max_depth':[5,6,7],
    'min_samples_split':[2,3,4],
    'min_samples_leaf': [3,4,5],
    'max_features':[None,"sqrt","log2"]}

In [45]:
DTR = GridSearchCV(DecisionTreeRegressor(),param_grid=params,cv=5,scoring='neg_root_mean_squared_error',n_jobs=-1)

In [328]:
DTR.fit(train_X,train_Y)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error'],
                         'max_depth': [5, 6, 7],
                         'max_features': [None, 'sqrt', 'log2'],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [2, 3, 4]},
             scoring='neg_root_mean_squared_error')

In [329]:
DTR.best_score_

-98059.76455496301

In [330]:
DTR.best_params_

{'criterion': 'squared_error',
 'max_depth': 5,
 'max_features': 'log2',
 'min_samples_leaf': 5,
 'min_samples_split': 3}

In [49]:
DTRF=DecisionTreeRegressor(criterion='squared_error',max_depth=5,max_features='sqrt',min_samples_leaf=5,min_samples_split=3)

In [332]:
DTRF.fit(train_X,train_Y)

DecisionTreeRegressor(max_depth=5, max_features='sqrt', min_samples_leaf=5,
                      min_samples_split=3)

###### Model Validation using RMSPE

In [333]:
trainpredval = pd.Series(DTRF.predict(train_X))

In [334]:
np.sqrt(np.mean(np.square(((train_Y-trainpredval)/train_Y)*100)))

2911.643423645129

In [335]:
testpredval = pd.Series(DTRF.predict(test_X))

In [336]:
np.sqrt(np.mean(np.square(((train_Y-testpredval)/train_Y)*100)))

2756.735049977024

##### Missing Values Predicted Output

In [53]:
clean_customer_topred.drop(columns='cc_cons',inplace=True)

C:\Users\Tarun\AppData\Local\Temp\ipykernel_10116\364805491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_customer_topred.drop(columns='cc_cons',inplace=True)


In [338]:
clean_customer_topred.columns

Index(['Avg_days_between_transaction', 'Emp_Tenure_Years', 'Income',
       'NetBanking_Flag_1', 'Tenure_with_Bank', 'account_type_saving', 'age',
       'cc_cons_apr', 'cc_cons_jun', 'cc_cons_may',
       ...
       'region_code_957_0', 'region_code_958_0', 'region_code_963_0',
       'region_code_964_0', 'region_code_965_0', 'region_code_966_0',
       'region_code_968_0', 'region_code_969_0', 'region_code_970_0',
       'region_code_974_0'],
      dtype='object', length=249)

In [339]:
clean_customer_topred = clean_customer_topred.reindex(sorted(clean_customer_topred.columns),axis=1)

In [340]:
prediction_value = pd.Series(DTRF.predict(clean_customer_topred))

In [341]:
prediction_value

0       63748.177287
1       63748.177287
2       48014.278628
3       48014.278628
4       63748.177287
            ...     
4995    48014.278628
4996    54896.734089
4997    54896.734089
4998    54896.734089
4999    54896.734089
Length: 5000, dtype: float64